## Trip Advisor Scraper

### Gardens by the Bay

In [2]:
from bs4 import BeautifulSoup
from lxml import html
import requests
import pandas as pd
import re

In [58]:
#base_url = "https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-Gardens_by_the_Bay-Singapore.html"
#base_url = "https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or2000-Gardens_by_the_Bay-Singapore.html#REVIEWS"
#base_url = "https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or4000-Gardens_by_the_Bay-Singapore.html#REVIEWS"
#base_url = "https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or6000-Gardens_by_the_Bay-Singapore.html#REVIEWS"
base_url = "https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19000-Gardens_by_the_Bay-Singapore.html#REVIEWS"

In [29]:
def parse(url):
    global next_page
    global counter
   
    urls = []
    url_list = []

    counter = counter + 1

    page = requests.get(url, params = {"limit": 1000, "offset": 0})
    soup = BeautifulSoup(page.text, "lxml")


#- Extract Review URLs from page(url)
    parent1 = soup.find_all("div", attrs = {"class":"reviewSelector   track_back"})
    parent2 = soup.find_all("div", attrs = {"class":"reviewSelector  "})
    
    for k in parent1: urls.append(k.find("div", attrs = {"class":"wrap"}).div.a.get("href"))
    for l in parent2: urls.append(l.find("div", attrs = {"class":"wrap"}).div.a.get("href"))
        
    urls = ["https://www.tripadvisor.com.sg" + x for x in urls]
    
    
#- Get next page URL
    next_page = soup.find("a", attrs = {"class":"nav next rndBtn ui_button primary taLnk"}).get("href")
    next_page = "https://www.tripadvisor.com.sg" + next_page

    
#- For each review get the details
    taReviewData = pd.DataFrame()

    for url in urls:
        review_page = requests.get(url, params = {"limit": 1000, "offset": 0})
        soup_review_page = BeautifulSoup(review_page.text, "lxml")
        
        
        review_id = soup_review_page.find("div", attrs = {"class":"  reviewSelector "}).get("id")
        review_title = soup_review_page.find("div", attrs = {"class":"quote"}).text
        review_text = soup_review_page.find("div", attrs = {"class":"entry"}).text
        
        try:
#            review_rating = soup_review_page.find("span", attrs = {"class":"rate sprite-rating_s rating_s"}).img.get("alt")
            review_rating = soup_review_page.find("div", attrs = {"id":"SHOW_USER_REVIEW"}).find("div", attrs = {"class":"rating reviewItemInline"}).find("span").get("alt")
        except:
            review_rating = "NoRating"
            
        regex = re.compile(r"([1-5])\.[0]\s")
        review_rating = regex.match(str(review_rating)).group(1)
       
        
        try:
            review_date = soup_review_page.find("span", attrs = {"class":"ratingDate relativeDate"}).get("title")
        except:
            try:
                review_date = soup_review_page.find("span", attrs = {"class":"ratingDate"}).get("content")
            except:
                review_date = "NoReviewDate"
                
        
        visit_date = soup_review_page.find("span", attrs = {"class":"recommend-titleInline noRatings"}).text
        regex = re.compile(r"Visited\s(.*)")
        visit_date = regex.match(str(visit_date)).group(1)
        
        
        try:
            reviewer_country = soup_review_page.find("div", attrs = {"class":"member_info"}).find("div", attrs = {"class":"location"}).text
        except:
            reviewer_country = " NoCity, NoCountry "
        
        regex = re.compile(r"[\s]?(.*)")
        try:
            reviewer_country = regex.match(str(reviewer_country[1:-1])).group(1)
        except:
            reviewer_country = "NoCountry"
            
            

        reviewer_level = soup_review_page.find("div", attrs = {"class":"memberBadging g10n"}).div.div.get("class")[-1]
        regex = re.compile(r"lvl_(.*)")
        try:
            reviewer_level = regex.match(str(reviewer_level)).group(1)
        except:
            reviewer_level = 0

        taReviewData = taReviewData.append(pd.DataFrame({"0review_id":review_id,
                                                         "1review_title":review_title,
                                                         "2review_text":review_text[2:-1],
                                                         "3review_rating":review_rating,
                                                         "4review_date":review_date,
                                                         "5visit_date":visit_date,
                                                         "6reviewer_country":reviewer_country,
                                                         "7reviewer_level":reviewer_level}, index=[0]),
                                                       ignore_index = True)
        
        
    return taReviewData
        

        


In [30]:
base_url

'https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or10000-Gardens_by_the_Bay-Singapore.html#REVIEWS'

In [59]:
df = pd.DataFrame()
counter = 0

df = parse(base_url)


while next_page and counter < 100:
    print(counter)
    print(next_page)
    df = df.append(parse(next_page), ignore_index=True)

df.to_csv("extract2_1900_1999.csv", header=True, index=False, encoding='utf-8')

1
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19010-Gardens_by_the_Bay-Singapore.html#REVIEWS
2
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19020-Gardens_by_the_Bay-Singapore.html#REVIEWS
3
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19030-Gardens_by_the_Bay-Singapore.html#REVIEWS
4
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19040-Gardens_by_the_Bay-Singapore.html#REVIEWS
5
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19050-Gardens_by_the_Bay-Singapore.html#REVIEWS
6
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19060-Gardens_by_the_Bay-Singapore.html#REVIEWS
7
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19070-Gardens_by_the_Bay-Singapore.html#REVIEWS
8
https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-or19080-Gardens_by_the_Bay-Singapore.html#

###### Points to check:

As with the links pages, we want to avoid downloading a given page the next time we need to run the notebook.  While we could save the files by hand, as we did before using dill, a checkpointing library like [ediblepickle](https://pypi.python.org/pypi/ediblepickle/1.1.3) can handle this for you.  (Note, though, that you may not want to enable this until you are sure that your function is working.)

You should also keep in mind that HTTP requests fail occasionally, for transient reasons.  You should plan how to detect and react to these failures.   The [retrying module](https://pypi.python.org/pypi/retrying) is one way to deal with this.

NLP:
* [nltk does part of speech tagging](http://www.nltk.org/book/ch05.html) but it is a little slow. 
* There may also be heuristics that accomplish the same thing.
* spaCy's [entity recognition](https://spacy.io/docs/usage/entity-recognition) couble be useful here.
* coreNLP

PageRank:
A similar way to determine popularity is to look at their
[pagerank](http://en.wikipedia.org/wiki/PageRank).  Pagerank is used for web ranking and was originally
[patented](http://patft.uspto.gov/netacgi/nph-Parser?patentnumber=6285999) by Google and is essentially the stationary distribution of a [markov
chain](http://en.wikipedia.org/wiki/Markov_chain) implied by the social graph.